<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/05-Improve_Prompts_%2B_Add_Source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables

In [ ]:
!pip install -q llama-index==0.9.21 openai==1.6.0 cohere==4.39 tiktoken==0.5.2 chromadb==0.4.21 kaleido==0.2.1 python-multipart==0.0.6 html2text==2020.1.16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.6/508.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"

In [ ]:
import nest_asyncio

nest_asyncio.apply()

# Load a Model

In [ ]:
from llama_index.llms import OpenAI

llm = OpenAI(temperature=0.9, model="gpt-3.5-turbo", max_tokens=512)

# Create a VectoreStore

In [ ]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = chroma_client.create_collection("mini-llama-articles")

In [ ]:
from llama_index.vector_stores import ChromaVectorStore

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Load the Dataset (CSV)

## Download

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model.

In [ ]:
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

--2024-02-01 14:53:37--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173646 (170K) [text/plain]
Saving to: ‘mini-llama-articles.csv’

mini-llama-articles 100%[===================>] 169.58K  --.-KB/s    in 0.02s   

2024-02-01 14:53:37 (6.64 MB/s) - ‘mini-llama-articles.csv’ saved [173646/173646]



## Load the Articles

In [ ]:
import csv

rows = []

# Load the file as a JSON
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
  csv_reader = csv.reader(file)

  for idx, row in enumerate( csv_reader ):
    if idx == 0: continue; # Skip header row
    rows.append( row )

# The number of characters in the dataset.
len( rows )

14

In [ ]:
rows[0][3]

'towards_ai'

# Convert to Document obj

In [ ]:
from llama_index import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=row[1], metadata={"title": row[0], "url": row[2], "source_name": row[3]}) for row in rows]

In [ ]:
len( documents )

14

# Transforming

In [ ]:
from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)

In [ ]:
from llama_index.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings import OpenAIEmbedding
from llama_index.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=3, llm=llm),
        SummaryExtractor(summaries=["prev", "self"], llm=llm),
        KeywordExtractor(keywords=10, llm=llm),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store
)

pipeline.run(documents=documents, show_progress=True)

Parsing nodes:   0%|          | 0/14 [00:00<?, ?it/s]

464
452
457
465
448
468
434
447
455
445
449
455
431
453


100%|██████████| 108/108 [00:48<00:00,  2.22it/s]


Generating embeddings:   0%|          | 0/108 [00:00<?, ?it/s]

# Load Indexes

In [ ]:
# Create your index
db = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = db.get_or_create_collection("mini-llama-articles")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [ ]:
# Create your index
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
res = query_engine.query("How many parameters LLaMA2 model has?")

In [ ]:
res.response

'The LLaMA2 model has four different parameter sizes: 7 billion, 13 billion, 34 billion, and 70 billion.'

In [ ]:
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("-_"*20)

Node ID	 cccc8c69-1648-469b-8a2a-ea7f003dbe27
Title	 Fine-Tuning a Llama-2 7B Model for Python Code Generation
Text	 New Llama-2 model In mid-July, Meta released its new family of pre-trained and finetuned models called Llama-2, with an open source and commercial character to facilitate its use and expansion. The base model was released with a chat version and sizes 7B, 13B, and 70B. Together with the models, the corresponding papers were published describing their characteristics and relevant points of the learning process, which provide very interesting information on the subject. For pre-training, 40% more tokens were used, reaching 2T, the context length was doubled and the grouped-query attention (GQA) technique was applied to speed up inference on the heavier 70B model. On the standard transformer architecture, RMSNorm normalization, SwiGLU activation, and rotatory positional embedding are used, the context length reaches 4096 tokens, and an Adam optimizer is applied with a cosin

# No Metadata

In [ ]:
from llama_index import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=row[1]) for row in rows]

In [ ]:
from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)

In [ ]:
from llama_index.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings import OpenAIEmbedding
from llama_index.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        text_splitter
    ]
)

nodes = pipeline.run(documents=documents, show_progress=True)

Parsing nodes:   0%|          | 0/14 [00:00<?, ?it/s]

510
510
510
510
510
510
510
510
510
510
510
510
510
510


In [ ]:
from llama_index import ServiceContext

index_no_metadata = VectorStoreIndex(
    nodes=nodes,
    service_context=ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo")),
)

In [ ]:
query_engine_no_metadata = index_no_metadata.as_query_engine()

In [ ]:
res = query_engine_no_metadata.query("How many parameters LLaMA2 model has?")

In [ ]:
res.response

'The context information does not provide any information about the number of parameters in the LLaMA2 model.'

In [ ]:
for src in res.source_nodes:
  print( src )
  print("-_"*20)

Node ID: 895debf4-60ad-4156-8f52-cdddf03b1138
Text: I. Llama 2: Revolutionizing Commercial Use Unlike its
predecessor Llama 1, which was limited to research use, Llama 2
represents a major advancement as an open-source commercial model.
Businesses can now integrate Llama 2 into products to create AI-
powered applications. Availability on Azure and AWS facilitates fine-
tuning and adoption. However,...
Score:  0.852

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID: a919ca6a-bdda-4d92-a7b2-ab4048cdc0d8
Text: basis. The sharing of codes and weights allows other researchers
to test new approaches in LLMs. The LLaMA models have a range of 7
billion to 65 billion parameters. LLaMA-65B can be compared to
DeepMind's Chinchilla and Google's PaLM. Publicly available unlabeled
data was used to train these models, and training smaller foundational
models requ...
Score:  0.830

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
